In [ ]:
import zipfile
import subprocess
import os, shutil
import gdal, osr
from dlfile import dlfile

webfolder = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/fews/web/africa/east/pentadal/emodis/ndvi/temporallysmoothedndvi/downloads/pentadal/'
for yr in range(16,17):
    for filenum in range(46,73):
        if filenum<10:
            filenumstr = '0'+str(filenum)
        else:
            filenumstr = str(filenum)
        #download file from web
        filename = 'ea'+str(yr)+filenumstr
        print(filename)
        fullurl = webfolder + filename + '.zip'
        print('Downloading file...')
        dlfile(fullurl)

        #unzip file
        zip_ref = zipfile.ZipFile(filename+'.zip') # create zipfile object
        print('Unzipping file...')
        zip_ref.extractall(filename) # extract file to folder with same filename
        zip_ref.close() # close file
        
        #check if has coordinate system and update if not
        unzipped_path = filename + '/' + filename + '.tif'
        new_path = filename + '/' + filename + 'm.tif'
        file_ds = gdal.Open(unzipped_path)
        file_srs = file_ds.GetProjectionRef()
        if 'unnamed' in file_srs:
            # create file with new coordinate_sys by reprojecting from wgs84 to wgs84
            os.system('gdalwarp -s_srs ''WGS84'' -t_srs ''WGS84'' "%s" "%s"' %(unzipped_path,new_path))
            print('File created with new coordinate system')
            new_ds = gdal.Open(new_path)
            new_srs = new_ds.GetProjectionRef()
            new_ds = None
        else:
            new_path = unzipped_path
        file_ds = None

        #upload file to google storage bucket
        print('Uploading to Google Storage bucket...')
        os.system('sh upload_to_gs.sh %s' %new_path)

        #delete zip and extracted files
        print('Deleting .zip and folder...')
        os.remove(filename+'.zip')
        shutil.rmtree(filename)
        print('')

In [ ]:
from datetime import datetime, timedelta
import calendar

gee_username = 'sinark'
gs_bucket = 'wheatrust'
#grant AWS earthengine account access to GS wheatrust files
access_str = 'gsutil acl ch -u sina.rk@gmail.com:OWNER gs://wheatrust/*.tif'
os.system(access_str)

eModis_cal = { #https://earlywarning.usgs.gov/fews/product/116
    1:['Dec 26','Jan 5'],  
    2:['Jan 1','Jan 10'],  
    3:['Jan 6','Jan 15'],
    4:['Jan 11','Jan 20'], 
    5:['Jan 16','Jan 25'], 
    6:['Jan 21','Jan 31'],
    7:['Jan 26','Feb 5'],  
    8:['Feb 1','Feb 10'],  
    9:['Feb 6','Feb 15'],
    10:['Feb 11','Feb 20'],
    11:['Feb 16','Feb 25'],
    12:['Feb 21','Feb 28'],
    13:['Feb 26','Mar 5'], 
    14:['Mar 1','Mar 10'], 
    15:['Mar 6','Mar 15'],
    16:['Mar 11','Mar 20'],
    17:['Mar 16','Mar 25'],
    18:['Mar 21','Mar 31'],
    19:['Mar 26','Apr 5'], 
    20:['Apr 1','Apr 10'], 
    21:['Apr 6','Apr 15'],
    22:['Apr 11','Apr 20'],
    23:['Apr 16','Apr 25'],
    24:['Apr 21','Apr 30'],
    25:['Apr 26','May 5'],
    26:['May 1','May 10'],
    27:['May 6','May 15'],
    28:['May 11','May 20'],
    29:['May 16','May 25'],
    30:['May 21','May 31'],
    31:['May 26','Jun 5'],
    32:['Jun 1','Jun 10'],
    33:['Jun 6','Jun 15'],
    34:['Jun 11','Jun 20'],
    35:['Jun 16','Jun 25'],
    36:['Jun 21','Jun 30'],
    37:['Jun 26','Jul 5'],
    38:['Jul 1','Jul 10'],
    39:['Jul 6','Jul 15'],
    40:['Jul 11','Jul 20'],
    41:['Jul 16','Jul 25'],
    42:['Jul 21','Jul 31'],
    43:['Jul 26','Aug 5'],
    44:['Aug 1','Aug 10'],
    45:['Aug 6','Aug 15'],
    46:['Aug 11','Aug 20'],
    47:['Aug 16','Aug 25'],
    48:['Aug 21','Aug 31'],
    49:['Aug 26','Sep 5'],
    50:['Sep 1','Sep 10'],
    51:['Sep 6','Sep 15'],
    52:['Sep 11','Sep 20'],
    53:['Sep 16','Sep 25'],
    54:['Sep 21','Sep 30'],
    55:['Sep 26','Oct 5'],
    56:['Oct 1','Oct 10'],
    57:['Oct 6','Oct 15'],
    58:['Oct 11','Oct 20'],
    59:['Oct 16','Oct 25'],
    60:['Oct 21','Oct 31'],
    61:['Oct 26','Nov 5'],
    62:['Nov 1','Nov 10'],
    63:['Nov 6','Nov 15'],
    64:['Nov 11','Nov 20'],
    65:['Nov 16','Nov 25'],
    66:['Nov 21','Nov 30'],
    67:['Nov 26','Dec 5'],
    68:['Dec 1','Dec 10'],
    69:['Dec 6','Dec 15'],
    70:['Dec 11','Dec 20'],
    71:['Dec 16','Dec 25'],
    72:['Dec 21','Dec 31'],
}

for yr in range(16,17):
    for filenum in range(52,73):
        if filenum<10:
            filenumstr = '0'+str(filenum)
        else:
            filenumstr = str(filenum)
        if yr==16 and filenum>51:
            filename = 'ea'+str(yr)+filenumstr #no need to reproject after this point
        else:
            filename = 'ea'+str(yr)+filenumstr+'m'
        print(filename)
        if filenum==1:
            yr_str = str(yr-1)
        else:
            yr_str = str(yr)            
        start = datetime.strptime(eModis_cal[filenum][0]+' 20'+yr_str,'%b %d %Y')
        end = datetime.strptime(eModis_cal[filenum][1]+' 20'+str(yr),'%b %d %Y')
        if filenum==12 and calendar.isleap(2000+yr):
            end = end+timedelta(days=1)
        start = start.strftime('%Y-%m-%d')
        end = end.strftime('%Y-%m-%d')
        yr_str = str(2000+yr)
        reg_params = '--property="(number)year=%s" --property="(number)pentad=%.0f"' %(yr_str,filenum)
        time_params = '--time_start %s --time_end %s' %(start,end)
        upload_str = 'earthengine upload image --asset_id=users/%s/eModisNDVI/%s %s %s gs://%s/%s.tif' \
            %(gee_username,filename,reg_params,time_params,gs_bucket,filename)
        print(upload_str)
        os.system(upload_str)